https://adventofcode.com/2021/day/8

In [1]:
from itertools import combinations
from collections import defaultdict, Counter

from toolz import concat

In [2]:
with open('data/08.txt') as fh:
    data = [x.strip() for x in fh]

In [3]:
def getdisplay(line):
    return line.strip().split('|')[1].split()

In [4]:
def count1478(line):
    return sum(len(x) in {2, 4, 3, 7} for x in getdisplay(line))

In [5]:
sum(count1478(line) for line in data)

247

In [6]:
def parseline(line):
    digits_str, display_str = line.strip().split('|')
    digits = [set(x) for x in digits_str.split()]
    display = [set(x) for x in display_str.split()]
    return (digits, display)

In [7]:
refdigits = [set(x) for x in ['abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg']]

In [8]:
refdigitlookup = {frozenset(x): str(i) for (i, x) in enumerate(refdigits)}

In [9]:
refdigitlookup

{frozenset({'a', 'b', 'c', 'e', 'f', 'g'}): '0',
 frozenset({'c', 'f'}): '1',
 frozenset({'a', 'c', 'd', 'e', 'g'}): '2',
 frozenset({'a', 'c', 'd', 'f', 'g'}): '3',
 frozenset({'b', 'c', 'd', 'f'}): '4',
 frozenset({'a', 'b', 'd', 'f', 'g'}): '5',
 frozenset({'a', 'b', 'd', 'e', 'f', 'g'}): '6',
 frozenset({'a', 'c', 'f'}): '7',
 frozenset({'a', 'b', 'c', 'd', 'e', 'f', 'g'}): '8',
 frozenset({'a', 'b', 'c', 'd', 'f', 'g'}): '9'}

In [18]:
def getlengthlookup(digits):
    D = {}
    for digit in digits:
        D.setdefault(len(digit), []).append(digit)
    return D      

In [19]:
reflengthlookup = getlengthlookup(refdigits)

reflengthlookup

{6: [{'a', 'b', 'c', 'e', 'f', 'g'},
  {'a', 'b', 'd', 'e', 'f', 'g'},
  {'a', 'b', 'c', 'd', 'f', 'g'}],
 2: [{'c', 'f'}],
 5: [{'a', 'c', 'd', 'e', 'g'},
  {'a', 'c', 'd', 'f', 'g'},
  {'a', 'b', 'd', 'f', 'g'}],
 4: [{'b', 'c', 'd', 'f'}],
 3: [{'a', 'c', 'f'}],
 7: [{'a', 'b', 'c', 'd', 'e', 'f', 'g'}]}

In [20]:
def getsegfreqlookup(digits):
    segfreqs = Counter(concat(digits))
    freqlookup = {}
    for seg, freq in segfreqs.items():
        freqlookup.setdefault(freq, set()).add(seg)
    return freqlookup

In [21]:
reffreqlookup = getsegfreqlookup(refdigits)
reffreqlookup

{8: {'a', 'c'}, 7: {'d', 'g'}, 9: {'f'}, 4: {'e'}, 6: {'b'}}

In [14]:
def makesetpairs(digits):
    pairs = []
    segfreqs = getsegfreqlookup(digits)
    for freq, refs in reffreqlookup.items():
        pairs.append((refs, segfreqs[freq]))
    return pairs

In [15]:
def decode(digits):
    pairs = makesetpairs(digits)
    lengthlookup = getlengthlookup(digits)
    seglookup = {}
    still_unknown = {}
    for x, y in pairs:
        if len(x) == 1:
            seglookup[next(iter(x))] = next(iter(y))
        else:
            for x1 in x:
                still_unknown[x1] = y.copy()
    seglookup['c'] = lengthlookup[2][0].copy().difference({seglookup['f']}).pop()
    seglookup['a'] = still_unknown['a'].copy().difference({seglookup['c']}).pop()    
    seglookup['d'] = lengthlookup[4][0].copy().difference({seglookup['b'], seglookup['c'], seglookup['f']}).pop()
    seglookup['g'] = still_unknown['g'].copy().difference({seglookup['d']}).pop()
    return {v:k for (k, v) in seglookup.items()}

In [16]:
def decode_display(digits, display):
    decoded = decode(digits)
    numbers = []
    for digit in display:
        decoded_digit = frozenset(decoded[x] for x in digit)
        numbers.append(refdigitlookup[decoded_digit])
    return int(''.join(numbers))
        

In [17]:
sum(decode_display(*parseline(line)) for line in data)

933305